In [1]:
using DynamicPolynomials, SparseArrays
println("***Problem setting***")

n=20
println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

function generate_sparse_random_poly(v)
    c=sprand(Float64,length(v),0.01)
    I,V=findnz(c)
    V=2*V.-1
    return sparsevec(I,V,length(v))'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:2))
f=generate_random_poly(v)


# unit sphere constraint
m=ceil(Int64, n/5)
g=Vector{Polynomial{true,Float64}}(undef,m)
 #type of coefficients of each polynomial must be float

randx=rand(n)# create a feasible solution
randx=rand(1)[1]*randx./sum(randx)
g[1]=1.0-sum(x)
for j in 2:m
    g[j]=generate_random_poly(v[2:end])
    g[j]+=0.125-g[j](x => randx) #make constraints feasible
end



m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64,n/5)

h=Vector{Polynomial{true,Float64}}(undef,l)
for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

#h=[h;1.0-sum(x.^2)]

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=20
Number of inequality constraints: m=4
Number of equality constraints: l=4


In [11]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

#n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);
include("./densePOPvar$(n)nineq$(m)neq$(l).jl");

k=1

println("Relaxation order: k=",k)

s=7

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false);

Relaxation order: k=1
Block size: s=7


Maximal matrix size:7
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1771            
  Cones                  : 0               
  Scalar variables       : 1688            
  Matrix variables       : 330             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : 

In [7]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

#n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);
include("./densePOPvar$(n)nineq$(m)neq$(l).jl");

k=1

println("Relaxation order: k=",k)

s=1

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=true);

Relaxation order: k=1
Block size: s=1


Maximal matrix size:1
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 5456            
  Cones                  : 0               
  Scalar variables       : 6294            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : LO

In [92]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);
#include("./densePOPvar$(n)nineq$(m)neq$(l).jl");

using TSSOS

k=2

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterruptedRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 231
************************TSSOS************************


TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[20]
[1]
------------------------------------------------------
Assembling the SDP...
There are 10626 affine constraints.
SDP assembling time: 0.862381407 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10626           
  Cones                  : 0               
  Scalar variables       : 925             
  Matrix variables       : 25              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1    

In [ ]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

InterruptedRelax.save_info_densePOP(randx,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)